# HST benchmark
In this notebook, we attempt to replicate the results presented in the [paper introducing Half-Space Trees](https://www.ijcai.org/Proceedings/11/Papers/254.pdf). The authors claim a .996 ROCAUC on the [KDD99HTTP dataset](http://odds.cs.stonybrook.edu/http-kddcup99-dataset/). We use `creme`'s implementation and attempt to replicate their results. We find that these results are only possible when the dataset is _prescaled_ and _shuffled,_ both options that invalidate the online learning paradigm.

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import time

from creme import datasets
from creme import anomaly
from creme import stream

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

This notebook runs using the API of `creme 0.4.4`

In [2]:
import creme
assert creme.__version__ == "0.4.4"

In [3]:
counter = 0
hst = anomaly.HalfSpaceTrees(
    n_trees=25,
    tree_height=15,
    window_size=250,
    scale=True
)

y_true = []
y_pred = []
start_time = time.time()
for x, y in datasets.fetch_kdd99_http():
    if counter%10000 == 0 and counter > 1:
        print(f"iteration: {counter}; time elapsed: {time.time()-start_time}")
        if np.mean(y_true) != 0:
            print(f"rocauc: {roc_auc_score(y_true, y_pred)}")
        else:
            print(f"rocauc: have only encountered one class")
    if counter > 250:
        y_true.append(y)
        y_pred.append(hst.score_one(x))
    hst.fit_one(x)
    counter += 1

iteration: 10000; time elapsed: 54.25441861152649
rocauc: have only encountered one class
iteration: 20000; time elapsed: 95.78404021263123
rocauc: have only encountered one class
iteration: 30000; time elapsed: 137.49432349205017
rocauc: have only encountered one class
iteration: 40000; time elapsed: 180.1520709991455
rocauc: have only encountered one class
iteration: 50000; time elapsed: 221.4606146812439
rocauc: have only encountered one class
iteration: 60000; time elapsed: 265.7636981010437
rocauc: have only encountered one class
iteration: 70000; time elapsed: 308.3105869293213
rocauc: have only encountered one class
iteration: 80000; time elapsed: 350.1188282966614
rocauc: have only encountered one class
iteration: 90000; time elapsed: 392.0654225349426
rocauc: have only encountered one class
iteration: 100000; time elapsed: 436.07345271110535
rocauc: have only encountered one class
iteration: 110000; time elapsed: 478.143785238266
rocauc: have only encountered one class
iterati

In [4]:
roc_auc_score(y_true, y_pred)

0.5048194989550726

In [5]:
roc_auc_score(y_true, y_pred)

0.5048194989550726

# Improper Preprocessing
We suspect that the authors of the HST paper rescaled and shuffled their data. Testing that:


In [7]:
import h5py
arrays = {}
f = h5py.File("http.mat")
for k,v in f.items():
    arrays[k] = np.array(v)

In [13]:
X0 = arrays["X"][0]
X1 = arrays["X"][1]
X2 = arrays["X"][2]
Y = arrays["y"]

In [18]:
X = pd.DataFrame(arrays["X"]).T

In [19]:
y = pd.DataFrame(arrays["y"]).astype(int).T

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [24]:
X["labels"] = Y[0].astype(int)

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
ss = StandardScaler()
X[[0,1,2]] = ss.fit_transform(X[[0,1,2]])

In [27]:
X_processed = X.sample(frac=1).reset_index(drop=True)

In [28]:
X_processed

,0,1,2,labels
0,-0.073165,-0.166589,1.403759,0
1,-0.073165,-0.244161,-1.313616,0
2,-0.073165,-0.324442,-0.970723,0
3,-0.073165,-0.304107,-0.383713,0
4,-0.073165,-0.073166,-1.019515,0
...,...,...,...,...
567493,-0.073165,0.664470,1.240416,0
567494,-0.073165,0.298008,1.499868,0
567495,-0.073165,-0.064029,-1.466999,0
567496,-0.073165,-0.629802,-1.407824,0


In [29]:
counter = 0
hst = anomaly.HalfSpaceTrees(
    n_trees=25,
    tree_height=15,
    window_size=250,
    scale=False
)

y_true = []
y_pred = []
start_time = time.time()
for x, y in stream.iter_pandas(X_processed[[0,1,2]], X_processed["labels"]):
    if counter%10000 == 0 and counter > 1:
        print(f"iteration: {counter}; time elapsed: {time.time()-start_time}")
        if np.mean(y_true) != 0:
            print(f"rocauc: {roc_auc_score(y_true, y_pred)}")
        else:
            print(f"have only encountered one class")
    if counter > 250:
        y_true.append(y)
        y_pred.append(hst.score_one(x))
    hst.fit_one(x)
    counter += 1

iteration: 10000; time elapsed: 75.02194046974182
rocauc: 0.006881556130107676
iteration: 20000; time elapsed: 127.55825424194336
rocauc: 0.007178005217248533
iteration: 30000; time elapsed: 185.54238057136536
rocauc: 0.007326296308796569
iteration: 40000; time elapsed: 244.52945280075073
rocauc: 0.006951168967740063
iteration: 50000; time elapsed: 300.64176630973816
rocauc: 0.007255643416195539
iteration: 60000; time elapsed: 360.61462688446045
rocauc: 0.007414315937197017
iteration: 70000; time elapsed: 412.75404119491577
rocauc: 0.010788325739034004
iteration: 80000; time elapsed: 462.4446954727173
rocauc: 0.010369131738800393
iteration: 90000; time elapsed: 507.32474851608276
rocauc: 0.009912449786697558
iteration: 100000; time elapsed: 560.4726068973541
rocauc: 0.009757245094066565
iteration: 110000; time elapsed: 607.2813560962677
rocauc: 0.009486988491269794
iteration: 120000; time elapsed: 650.6357972621918
rocauc: 0.009205968371137753
iteration: 130000; time elapsed: 694.37321

In [30]:
roc_auc_score(y_true, y_pred)

0.00864682372900627